# BerlinMOD Queries

So far we have replicated the BerlinMOD Pymeos tutorial using Pyspark. Now we will execute the BerlinMOD queries.

In [1]:
cd "../mobilitydb-berlinmod-sf0.1/"

/data/mobilitydb-berlinmod-sf0.1


/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
ls

instants.csv*  points.csv*       trips.csv*       vehicles.csv*
licences.csv*  regions.csv*      trips_small.csv  vehicles_small.csv
periods.csv*   spark-warehouse/  vehicle_ids.txt


In [3]:
rm -R spark-warehouse/

## Imports

In [4]:
from pymeos import *
from pymeos.plotters import *

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pysparkmeos.UDT.MeosDatatype import *
from pysparkmeos.partitions.grid.grid_partitioner import GridPartition
from pysparkmeos.utils.udt_appender import udt_append
from pysparkmeos.utils.utils import register_udfs_under_spark_sql


from pysparkmeos.UDF.udf import *
from pysparkmeos.partitions.mobilityrdd import MobilityRDD


import random, datetime, os, sys
from datetime import timedelta
from functools import partial
from datetime import datetime, timezone
import contextily as cx
import distinctipy
import geopandas as gpd
import pandas as pd
import shapely.geometry as shp

import matplotlib.pyplot as plt
import numpy as np
from shapely import wkb, box, from_wkb
from typing import Union
from time import time

## Spark Initialization

In [5]:
def startspark():
    # Initialize PyMEOS
    pymeos_initialize("UTC")
    
    os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
    os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
    os.environ['PYSPARK_PYTHON'] = sys.executable
    
    #.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    # Initialize a Spark session
    spark = SparkSession.builder \
        .appName("PySpark UDF Example with PyMEOS") \
        .master("local[*]") \
        .config("spark.default.parallelism", 50) \
        .config("spark.executor.memory", "4g") \
        .config("spark.driver.memory", "4g") \
        .getOrCreate()
    
    # spark.sparkContext.setLogLevel("INFO")
    
    # Append the UDT mapping to the PyMEOS classes
    udt_append()
    
    # Get the value of 'spark.default.parallelism'
    default_parallelism = spark.sparkContext.getConf().get("spark.default.parallelism")
    print(f"spark.default.parallelism: {default_parallelism}")

    # Register udfs in Spark SQL
    register_udfs_under_spark_sql(spark)
    
    return spark

spark = startspark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/08 19:46:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/08 19:46:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


spark.default.parallelism: 50


## Load Tables
We will use the power of Spark SQL to read in the raw dataframes and then create the tables.

In [6]:
ls

instants.csv*  periods.csv*  regions.csv*  trips_small.csv  vehicles.csv*
licences.csv*  points.csv*   trips.csv*    vehicle_ids.txt  vehicles_small.csv


In [7]:
def load_table(spark, path, tablename, partition_key=None, transformation_query=None, **kwargs):
    print("Reading raw csv ", path)
    rawdf = spark.read.csv(path, **kwargs)

    print("Creating temp view of raw table")
    rawdf.createOrReplaceTempView(f"{tablename}Raw")

    print("Schema and statistics of raw table")
    rawdf.printSchema()
    rawdf.describe().show()
    print(f"Creating final table {tablename} based on {tablename}Raw, partitioned by {partition_key}.")
    spark.sql(f"""DROP TABLE IF EXISTS {tablename}""")

    if transformation_query:
        rawdf = spark.sql(transformation_query)
        rawdf.createOrReplaceTempView(f"{tablename}Raw")

    start = time()
    
    if partition_key:
        df = spark.sql(f"""
        CREATE TABLE {tablename}
        USING parquet
        PARTITIONED BY ({partition_key})
        AS SELECT * FROM {tablename}Raw
        """)
    else:
        df = spark.sql(f"""
        CREATE TABLE {tablename}
        USING parquet
        AS SELECT * FROM {tablename}Raw
        """)
        
    end = time()
    
    if partition_key:
        print(f"{tablename} partitions:")
        spark.sql(f"""
        SHOW PARTITIONS {tablename}
        """).show()
    print(f"Final table created in {end-start} seconds")

    df = spark.sql(f"SELECT * FROM {tablename}")

    print(f"Final table {tablename} schema:")
    df.printSchema()
    
    #Drop the temporary view
    spark.catalog.dropTempView(f"{tablename}Raw")
    return df, (start, end, end-start)


def load_all_tables(configs):
    tables = {}
    stats = {}
    for tablename, config in configs.items():
        table, stat = load_table(**config)
        tables[tablename] = table
        stats[tablename] = stat
    return tables, stats

### Instants

In [8]:
#instants, statsinstants = load_table(spark, "instants.csv", 'instants', inferSchema=True, header=True)
#instants.show()

### Licences

In [9]:
#licences, statslicences = load_table(spark, "licences.csv", 'licences', inferSchema=True, header=True)
#licences.show()

### Periods

In [10]:
transperiod = """
SELECT periodid, beginp, endp, tstzspan(period) AS period FROM periodsRaw
"""
#periods, statsperiods = load_table(spark, "periods.csv", 'periods', transformation_query=transperiod, inferSchema=True, header=True)
#periods.show()

### Points

In [11]:
transpoints = """
SELECT pointid, posx, posy, geometry_from_hexwkb(geom) AS geom FROM pointsRaw
"""
#points, statspoints = load_table(spark, "points.csv", 'points', transformation_query=transpoints, inferSchema=True, header=True)
#points.show()

### Regions

In [12]:
transregions = "SELECT regionid, geometry_from_hexwkb(geom) AS geom FROM regionsRaw"
#regions, statsregions = load_table(spark, "regions.csv", 'regions', transformation_query=transregions, inferSchema=True, header=True)
#regions.show()

### Trips
Note: Use trips_small for testing.

In [13]:
transtrips = "SELECT tripid, vehid, day, seqno, sourcenode, targetnode, trip_from_hexwkb(trip) AS trip, geometry_from_hexwkb(trajectory) AS trajectory FROM tripsRaw"

#trips, statstrips = load_table(
#    spark, "trips_small.csv", 'trips', 
#    transformation_query=transtrips,
#    partition_key= 'vehid',
#    inferSchema=True, 
#    header=True
#)
#trips.show()

### Vehicles
Note: Also read vehicles_small for testing.

In [14]:
#vehicles, statsvehicles = load_table(spark, "vehicles_small.csv", 'vehicles', inferSchema=True, header=True)
#vehicles.show()

## Queries

First queries take a general approach and are only used to measure overall performance.

In [15]:
from time import time
def query_exec(query, spark, execute=True, explain=False, explainmode=''):
    plan = None
    if explain:
        plan = spark.sql(f"EXPLAIN {explainmode} {query}").collect()[0].plan
    result = spark.sql(query)
    start = time()
    if execute:
        result.show()
    end = time()
    print("Query execution time: ", end-start, " seconds.")
    return result, (start, end, end-start), plan


def retrieve_exec_stats(queries, starts, ends, durations, plans):
    return pd.DataFrame({"queries": queries, "start": starts, "end": ends, "duration": durations, "plan": plans})


def run_all_queries(queries, spark, execute=True, explain=True, explainmode='', printplan=False):
    """ Utility function to run all queries through subsequent experiments """
    qdfs = []
    starts = []
    ends = []
    durations = []
    plans = []
    for querytext in queries:
        qdf, qstats, plan = query_exec(querytext, spark, execute, explain, explainmode)
        qdfs.append(qdf)
        starts.append(qstats[0])
        ends.append(qstats[1])
        durations.append(qstats[2])
        plans.append(plan)
        if printplan:
            print(plan)
    exec_stats = retrieve_exec_stats(queries, starts, ends, durations, plans)
    return qdfs, exec_stats

### Query 1: What are the models of the vehicles with licence plate numbers from QueryLicences?

In [16]:
querytext1 = """
    SELECT l.licence, v.model
    FROM licences l, vehicles v
    WHERE l.licence = v.licence
"""
#q1, q1stats, plan1 = query_exec(querytext1, spark, explain=True)
#if plan1:
#    print(plan1)

### Query 2: How many vehicles exist that are 'passenger' cars?

In [17]:
querytext2 = """
    SELECT COUNT(licence) AS PassengerCarCount
    FROM vehicles
    WHERE type='passenger'
"""
#q2, q2stats, plan2 = query_exec(querytext2, spark, explain=True)
#if plan2:
#    print(plan2)

### Query 3: Where have the vehicles with licences from QueryLicences1 been at each of the instants from QueryInstants1?

In [18]:
from datetime import datetime

querytext3 = """
    WITH 
    veh_w_lic AS (
        SELECT v.vehid, l.licence, v.model
        FROM licences l, vehicles v
        WHERE l.licence = v.licence
    ),
    veh_trips AS (
        SELECT t.* 
        FROM veh_w_lic vw, trips t
        WHERE t.vehid = vw.vehid
    )
    SELECT vt.vehid, vt.tripid, vt.trip, i.instant, tpoint_at(vt.trip, i.instant) AS pos
    FROM veh_trips vt, instants i
    WHERE temporally_contains(vt.trip, i.instant) = TRUE
"""
#q3, q3stats, plan3 = query_exec(querytext3, spark, explain=True)
#if plan3:
#    print(plan3)

### Query 4: Which licence plate numbers belong to vehicles that have passed the points from QueryPoints?

In [19]:
querytext4 = """
    WITH 
    vehids_intersect AS (
        SELECT t.vehid
        FROM trips t, points p
        WHERE ever_touches(t.trip, p.geom) = TRUE
    )
    SELECT vi.vehid, v.licence
    FROM vehids_intersect vi, vehicles v
"""
#q4, q4stats, plan4 = query_exec(querytext4, spark, explain=True)
#if plan4:
#    print(plan4)

In [20]:
dummy_df = spark.createDataFrame(["0"], "string").toDF("temp_clm")
dummy_df.show()

24/05/08 19:46:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------+
|temp_clm|
+--------+
|       0|
+--------+



In [21]:
queries = [querytext1, querytext2, querytext3, querytext4]

## Experiments

### Experiment 1: Run Queries ASIS
First we are going to run the queries without any improvement or partitioning strategy.

#### Create Tables

In [22]:
rm -R spark-warehouse/

/bin/rm: cannot remove 'spark-warehouse/': No such file or directory


In [23]:
configs_exp1 = {
    'instants': {'spark': spark, 'path': 'instants.csv', 'tablename': 'instants', 'inferSchema': True, 'header': True},
    'licences': {'spark': spark, 'path': 'licences.csv', 'tablename': 'licences', 'inferSchema': True, 'header': True},
    'periods':  {'spark': spark, 'path': 'periods.csv', 'tablename': 'periods', 'transformation_query': transperiod, 'inferSchema': True, 'header': True},
    'points':   {'spark': spark, 'path': 'points.csv', 'tablename': 'points', 'transformation_query': transpoints, 'inferSchema': True, 'header': True},
    'regions':  {'spark': spark, 'path': 'regions.csv', 'tablename': 'regions', 'transformation_query':transregions, 'inferSchema': True, 'header': True},
    'trips':    {'spark': spark, 'path': 'trips_small.csv', 'tablename': 'trips', 'transformation_query':transtrips, 'inferSchema': True, 'header': True},
    'vehicles': {'spark': spark, 'path': 'vehicles_small.csv', 'tablename': 'vehicles', 'inferSchema': True, 'header': True}
}

In [24]:
tables, stats = load_all_tables(configs_exp1)

Reading raw csv  instants.csv


Creating temp view of raw table
Schema and statistics of raw table
root
 |-- instantid: integer (nullable = true)
 |-- instant: timestamp (nullable = true)



+-------+------------------+
|summary|         instantid|
+-------+------------------+
|  count|               100|
|   mean|              50.5|
| stddev|29.011491975882016|
|    min|                 1|
|    max|               100|
+-------+------------------+

Creating final table instants based on instantsRaw, partitioned by None.


Final table created in 3.2364754676818848 seconds
Final table instants schema:
root
 |-- instantid: integer (nullable = true)
 |-- instant: timestamp (nullable = true)

Reading raw csv  licences.csv
Creating temp view of raw table
Schema and statistics of raw table
root
 |-- licenceid: integer (nullable = true)
 |-- licence: string (nullable = true)
 |-- vehid: integer (nullable = true)



+-------+------------------+--------+------------------+
|summary|         licenceid| licence|             vehid|
+-------+------------------+--------+------------------+
|  count|               101|     101|               101|
|   mean|              51.0|    NULL|319.46534653465346|
| stddev|29.300170647967224|    NULL| 175.0106604956644|
|    min|                 1|B-BJ 115|                 9|
|    max|               101|B-[U 177|               622|
+-------+------------------+--------+------------------+

Creating final table licences based on licencesRaw, partitioned by None.


...

Final table created in 3.018003225326538 seconds
Final table points schema:
root
 |-- pointid: integer (nullable = true)
 |-- posx: double (nullable = true)
 |-- posy: double (nullable = true)
 |-- geom: pythonuserdefined (nullable = true)

Reading raw csv  regions.csv
Creating temp view of raw table
Schema and statistics of raw table
root
 |-- regionid: integer (nullable = true)
 |-- geom: string (nullable = true)

+-------+------------------+--------------------+
|summary|          regionid|                geom|
+-------+------------------+--------------------+
|  count|               100|                 100|
|   mean|              50.5|                NULL|
| stddev|29.011491975882016|                NULL|
|    min|                 1|0103000020110F000...|
|    max|               100|0103000020110F000...|
+-------+------------------+--------------------+

Creating final table regions based on regionsRaw, partitioned by None.


Final table created in 2.9201602935791016 seconds
Final table regions schema:
root
 |-- regionid: integer (nullable = true)
 |-- geom: pythonuserdefined (nullable = true)

Reading raw csv  trips_small.csv
Creating temp view of raw table
Schema and statistics of raw table
root
 |-- tripid: integer (nullable = true)
 |-- vehid: integer (nullable = true)
 |-- day: date (nullable = true)
 |-- seqno: integer (nullable = true)
 |-- sourcenode: integer (nullable = true)
 |-- targetnode: integer (nullable = true)
 |-- trip: string (nullable = true)
 |-- trajectory: string (nullable = true)
 |-- licence: string (nullable = true)



+-------+-----------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+-------+
|summary|           tripid|             vehid|             seqno|       sourcenode|       targetnode|                trip|          trajectory|licence|
+-------+-----------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+-------+
|  count|               91|                91|                91|               91|               91|                  91|                  91|      0|
|   mean|304.3296703296703|10.956043956043956|2.5934065934065935|39454.89010989011|39454.89010989011|                NULL|                NULL|   NULL|
| stddev|204.9905176638967| 7.067786213065726| 1.666520140079134|28341.44503743612|28341.44503743612|                NULL|                NULL|   NULL|
|    min|                1|                 1|                 1|             1160|     

Final table created in 7.305429697036743 seconds
Final table trips schema:
root
 |-- tripid: integer (nullable = true)
 |-- vehid: integer (nullable = true)
 |-- day: date (nullable = true)
 |-- seqno: integer (nullable = true)
 |-- sourcenode: integer (nullable = true)
 |-- targetnode: integer (nullable = true)
 |-- trip: pythonuserdefined (nullable = true)
 |-- trajectory: pythonuserdefined (nullable = true)

Reading raw csv  vehicles_small.csv
Creating temp view of raw table
Schema and statistics of raw table
root
 |-- vehid: integer (nullable = true)
 |-- licence: string (nullable = true)
 |-- type: string (nullable = true)
 |-- model: string (nullable = true)



+-------+-----------------+-------+-----+--------+
|summary|            vehid|licence| type|   model|
+-------+-----------------+-------+-----+--------+
|  count|                3|      3|    3|       3|
|   mean|             11.0|   NULL| NULL|    NULL|
| stddev|8.717797887081348|   NULL| NULL|    NULL|
|    min|                1|B-CJ 17|  bus|    Opel|
|    max|               17|B-PZ 15|truck|Wartburg|
+-------+-----------------+-------+-----+--------+

Creating final table vehicles based on vehiclesRaw, partitioned by None.
Final table created in 1.6831741333007812 seconds
Final table vehicles schema:
root
 |-- vehid: integer (nullable = true)
 |-- licence: string (nullable = true)
 |-- type: string (nullable = true)
 |-- model: string (nullable = true)



In [25]:
@F.udtf(returnType="num: int, squared: int")
class SquareNumbers:
    def eval(self, start: int, end: int):
        for num in range(start, end + 1):
            yield (num, num * num)

SquareNumbers(F.lit(1), F.lit(3)).show()

+---+-------+
|num|squared|
+---+-------+
|  1|      1|
|  2|      4|
|  3|      9|
+---+-------+



In [26]:
"""
 |-- tripid: integer (nullable = true)
 |-- vehid: integer (nullable = true)
 |-- day: date (nullable = true)
 |-- seqno: integer (nullable = true)
 |-- sourcenode: integer (nullable = true)
 |-- targetnode: integer (nullable = true)
 |-- trip: pythonuserdefined (nullable = true)
 |-- trajectory: pythonuserdefined (nullable = true)
"""
schema = StructType([
    StructField("point", TGeomPointInstUDT())
])
@F.udtf(returnType=schema)
class ExplodeGeomSeq:
    def eval(self, trip: TGeomPointSeqWrap):
        #print(trip['trip'])
        #trip = trip.trip
        pymeos_initialize()
        instants = trip['trip'].instants()
        for i in instants:
            yield i,

spark.udtf.register("explodeGeomSeq", ExplodeGeomSeq)

In [27]:
spark.sql("SELECT * FROM explodeGeomSeq(TABLE(SELECT trip FROM trips))").show()

+--------------------+
|               point|
+--------------------+
|POINT(496253.8408...|
|POINT(496250.4704...|
|POINT(496231.2318...|
|POINT(496254.3235...|
|POINT(496254.4530...|
|POINT(496259.5870...|
|POINT(496262.1540...|
|POINT(496269.8550...|
|POINT(496270.4051...|
|POINT(496283.2368...|
|POINT(496283.6076...|
|POINT(496291.7883...|
|POINT(496291.7883...|
|POINT(496293.4705...|
|POINT(496310.5692...|
|POINT(496318.5173...|
|POINT(496320.9330...|
|POINT(496324.3406...|
|POINT(496331.1558...|
|POINT(496331.6642...|
+--------------------+
only showing top 20 rows



#### Execute Queries

In [ ]:
qdfs_exp1, stats_exp1 = run_all_queries(queries, spark, explain=True, printplan=True)

In [ ]:
for (_id, rdd) in spark.sparkContext._jsc.getPersistentRDDs().items():
    rdd.unpersist()
    print("Unpersisted {} rdd".format(_id))
spark.stop()

### Experiment 2: Partition Trips by vehid, HashPartitioning

In [ ]:
rm -R rm -R spark-warehouse/

In [ ]:
spark = startspark()

#### Create Tables

In [ ]:
configs_exp2 = {
    'instants': {'spark': spark, 'path': 'instants.csv', 'tablename': 'instants', 'inferSchema': True, 'header': True},
    'licences': {'spark': spark, 'path': 'licences.csv', 'tablename': 'licences', 'inferSchema': True, 'header': True},
    'periods':  {'spark': spark, 'path': 'periods.csv', 'tablename': 'periods', 'transformation_query': transperiod, 'inferSchema': True, 'header': True},
    'points':   {'spark': spark, 'path': 'points.csv', 'tablename': 'points', 'transformation_query': transpoints, 'inferSchema': True, 'header': True},
    'regions':  {'spark': spark, 'path': 'regions.csv', 'tablename': 'regions', 'transformation_query':transregions, 'inferSchema': True, 'header': True},
    'trips':    {'spark': spark, 'path': 'trips_small.csv', 'tablename': 'trips', 'partition_key': 'vehid', 'transformation_query':transtrips, 'inferSchema': True, 'header': True},
    'vehicles': {'spark': spark, 'path': 'vehicles_small.csv', 'tablename': 'vehicles', 'inferSchema': True, 'header': True}
}

In [ ]:
tables, stats = load_all_tables(configs_exp2)

#### Execute Queries

In [ ]:
qdfs_exp2, stats_exp2 = run_all_queries(queries, spark, explain=True, printplan=True)

In [ ]:
spark.stop()